In [1]:
import numpy as np
from sympy import symbols, cos, sin, pi, simplify
from sympy.matrices import Matrix
import pandas as pd

In [2]:
data2 = pd.read_csv('/Users/yusufdineh/Documents/GitHub/EE347/Lab2/mecharm_control_group_5.csv')
#Joint Angles from task 3
joint_angles = data2[['J1','J2','J3','J4','J5','J6']].to_numpy()
#End effector positions from task 3
test_end_effector_positions = data2[['X','Y','Z']].to_numpy()

In [3]:
#Global variables
speed = 30 # Speed for moving servos
sleepTime = 5 # Sleep time after moving servos
origin = [0, 0, 0, 0, 0, 0] # Origin position for servos
q1, q2, q3, q4, q5, q6 = symbols('q1 q2 q3 q4 q5 q6') # Define symbolic variables for joint angles
right = pi/2

In [4]:
# Error correction settings
max_retries = 3  # Number of times to retry sending angles if they don't match
angle_tolerance = 1.0  # Degrees tolerance for angle comparison
coord_tolerance = 5.0  # mm (or units returned by get_coords) tolerance for coordinate comparison

In [5]:
def get_transformation_matrix(a, alpha, d, theta):
    return Matrix([
        [cos(theta), -sin(theta)*cos(alpha), sin(theta)*sin(alpha), a*cos(theta)],
        [sin(theta), cos(theta)*cos(alpha), -cos(theta)*sin(alpha), a*sin(theta)],
        [0, sin(alpha), cos(alpha), d],
        [0, 0, 0, 1]
    ])

def overall_transformation(dh_table):
    T = Matrix(np.identity(4))
    for i in range(len(dh_table)):
        a, alpha, d, theta = dh_table[i]
        T_i = get_transformation_matrix(a, alpha, d, theta)
        T = T * T_i
    return simplify(T)

In [40]:
# Generate Empty DH table for MechArm
# DH table rows: [a, alpha, d, offset, offset]
# a      : link length (mm)
# alpha  : link twist (radians)
# d      : link offset (mm)
# offset : constant offset added to the joint variable (radians)
# q      : symbolic joint variable (q1..q6)
dh_table = [
    [0,         -right,       130,  q1],           # joint 1
    [105,           0,         0,   q2 - right],   # joint 2
    [10,        -right,       0,   q3],           # joint 3
    [0,         right,      100,   q4],           # joint 4
    [0,         -right,       0,   q5],           # joint 5
    [0,         0,         80,   q6]              # joint 6
]

In [41]:
# Calculate overall transformation matrix
T = overall_transformation(dh_table)
calculated_position = np.zeros((len(joint_angles), 3))
for i, q_vals in enumerate(joint_angles):
    # Convert joint angles to radians
    q1_val, q2_val, q3_val, q4_val, q5_val, q6_val = np.radians(q_vals)
    
    # Substitute joint angles into transformation matrix
    T_substituted = T.subs({q1: q1_val, q2: q2_val, q3: q3_val, q4: q4_val, q5: q5_val, q6: q6_val})
    end_effector_position = T_substituted[:3, 3]
    calculated_position[i, :] = np.array([float(end_effector_position[0]), float(end_effector_position[1]), float(end_effector_position[2])])

## Verifying the DH parameters using Task 3 data

We verified the Denavit–Hartenberg (DH) parameters by replaying joint angles gathered from Task 3 through our forward kinematics implementation and comparing the resulting end-effector positions to the measured test positions from the same dataset.

1. Data split

- We loaded the CSV exported from Task 3 and split it into two parts:
  - Joint angles: the columns `J1`..`J6` (these were used as inputs to the forward kinematics),
  - Measured end-effector positions: the `X`, `Y`, `Z` columns (these are the expected/ground-truth positions).

2. Forward kinematics run

- We built a symbolic forward kinematics chain from the DH table using SymPy.
- For each row of joint angles we converted degrees → radians, substituted the numeric values into the symbolic transform, and extracted the computed end-effector position (x, y, z).

3. Error calculation

- For each sample we computed the Euclidean distance between the measured position and the calculated position:

  error_i = ||p_measured_i - p_calculated_i||

- We then computed summary statistics (mean, max, min, standard deviation) over all samples.

4. Results (from the run below)

- Mean Error: 13.3512 mm
- Max Error: 17.3087 mm
- Min Error: 4.5770 mm
- Standard Deviation: 4.1314 mm

5. Why these errors occur

The observed errors are expected when you're not the robot manufacturer. Causes include:

- Manufacturer-specific offsets and non-idealities: exact link lengths, tool offsets, and joint zero positions are often proprietary or vary slightly from published specs.
- Assembly and manufacturing tolerances: small differences in joint placement, link machining, and fastener torques change the effective kinematics.
- Simplified DH model: the standard DH parameters assume ideal rigid links and perfect revolute joints; real robots have flex, backlash, and compliance.
- Sensor and measurement noise: encoder quantization, measurement noise when capturing X/Y/Z, and rounding when exporting CSV.
- Calibration/registration errors: coordinate frame misalignment between how the measurements were taken and how the DH frames were defined.

To reduce these errors you'd normally perform a calibration procedure (e.g., pose-based kinematic calibration) to estimate correction parameters, joint offsets, and tool transforms. Since we didn't have manufacturer's calibration data, the error above is an expected result.

Below we show the code that produced the numeric results (running the forward kinematics and computing per-sample errors).

In [42]:
# Calculate errors for my_dh_table
errors_my_dh_table = np.linalg.norm(calculated_position - test_end_effector_positions, axis=1)
mean_error_my_dh_table = np.mean(errors_my_dh_table)
max_error_my_dh_table = np.max(errors_my_dh_table)
min_error_my_dh_table = np.min(errors_my_dh_table)

print("Errors using my_dh_table:")
print(f"Mean Error: {mean_error_my_dh_table:.4f} mm")
print(f"Max Error: {max_error_my_dh_table:.4f} mm")
print(f"Min Error: {min_error_my_dh_table:.4f} mm")
print(f"Standard Deviation: {np.std(errors_my_dh_table):.4f} mm")


Errors using my_dh_table:
Mean Error: 13.3512 mm
Max Error: 17.3087 mm
Min Error: 4.5770 mm
Standard Deviation: 4.1314 mm
